# Lecture1 Sentence2vec

这次的research highlight是Princeton的一篇论文，主要内容是一个简单但很难超越的Sentence Embedding基线方法

在神经网络泛滥的时候，这篇文章像一股清流，提出了一个无监督的句子建模方法，并且给出了该方法的一些理论解释。通过该方法得到的句子向量，在STS数据集上取得了不输给神经网络的效果。

![1.jpg](https://i.loli.net/2018/04/10/5acc07189eebd.jpg)

### 句子Embedding动机

第二节课一直在讲词向量编码词的意思，但自然语言处理真正关心的是整个句子的意思。

如果我们能够拿到句子的向量表示，则可以方便的用內积计算相似度：

![2.jpg](https://i.loli.net/2018/04/10/5acc07c88995e.jpg)

还可以在这些句子向量之上构建分类器做情感分析：

![3.jpg](https://i.loli.net/2018/04/10/5acc07fabe878.jpg)

### 已有方法

具体怎么由词向量到句向量呢？有很多种方法，比如词袋模型中简单地线性运算：

![4.jpg](https://i.loli.net/2018/04/10/5acc0840e4102.jpg)

后面的课程中，将会用recurrent neural network、recursive neural network，CNN来做同样的事情。

![5.jpg](https://i.loli.net/2018/04/10/5acc08af83599.jpg)

### A Simple but Tough-to-beat Baseline for Sentence Embeddings

但今天要介绍的这篇普林斯顿大学的论文却剑走偏锋，采用了一种简单的无监督方法。这种方法简单到只有两步：

1. 对句子中的每个词向量，乘以一个独特的权值。这个权值是一个常数α除以α与该词语频率的和，也就是说高频词的权值会相对下降。求和后得到暂时的句向量s。
2. 计算语料库所有句向量构成的矩阵的第一个主成分u，让每个句向量减去它在u上的投影（类似PCA）。其中，一个向量v在另一个向量u上的投影定义如下：

	![屏幕快照 2018-04-10 上午8.46.10.png](https://i.loli.net/2018/04/10/5acc0962265af.png)
	
![6.jpg](https://i.loli.net/2018/04/10/5acc098c1c106.jpg)

### 句子建模算法

作者将该算法称之为WR，W表示Weighted，意为使用预估计的参数给句中的每个词向量赋予权重，R表示Removal，意为使用PCA或者SVD方法移除句向量中的无关部分。

![9.png](https://i.loli.net/2018/04/10/5acc11322a8a6.png)

输入： 
- 预训练的词向量{$v_{w}$:w∈V}，例如word2vec、glove等 
- 待处理的句子集合 S
- 参数a（论文中建议a的范围：[1e−4,1e−3]） 
- 词频估计{p(w):w∈V}

输出： 
- 句子向量{$v_{s}$:s∈S}


### 概率论解释

其原理是，给定上下文向量，一个词的出现概率由两项决定：作为平滑项的词频，以及上下文：

![7.jpg](https://i.loli.net/2018/04/10/5acc09f15faa2.jpg)

其中第二项的意思是，有一个平滑变动的上下文随机地发射单词。

### 效果

在句子相似度任务上超过平均水平，甚至超过部分复杂的模型。在句子分类上效果也很明显，甚至是最好成绩。

![8.jpg](https://i.loli.net/2018/04/10/5acc0a278ce29.jpg)

这是文中的第一个实验——句子相似性评价。 
实验使用的数据集都是公共数据集，在这些数据集上方法都取得了不输给RNN和LSTM的表现。

但是在情感分析方面，该方法不及RNN和LSTM，作者分析的了可能原因： 
1. 算法使用的词向量(word2vec, glove等)大都基于分布式假说——拥有相近上下文的单词具有相近的意思，但是这些词向量对句子中的antonym problem(我的理解是句子中会出现转折)的感知能力有限。 
2. 对于预估计词频来确定权重的方式，在情感分析中可能不是很有效。例如，单词”not”在情感分析中是非常重要的，但是在确定权重时，采用的词频估计会导致其难以在情感分析中发挥作用。

### 实验复现



In [ ]:
# -*- coding:utf8 -*-
from gensim.models import KeyedVectors
import pickle as pkl
import numpy as np
from typing import List
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import os
# import PSLvec as psl
from nltk.tokenize import StanfordTokenizer

word2vec_path = 'GoogleNews-vectors-negative300.bin.gz'
# glove_path = './glove_model.txt'
# psl_path = './PSL_model.txt'
# traindata = './datasets/sts2013.OnWN.pkl'
# freq_table = './mydictionary'
embedding_size = 300

# pslemb = psl.PSL()

# 载入word2vec模型
model = KeyedVectors.load_word2vec_format(word2vec_path,binary=True)
# model = KeyedVectors.load_word2vec_format(glove_path,binary=False)
# model = KeyedVectors.load_word2vec_format(psl_path,binary=False)
# model = pslemb.w
print('完成模型载入')

# tokenizer = StanfordTokenizer(path_to_jar=r"D:\stanford-parser-full-2016-10-31\stanford-parser.jar")


# print(type(model))
# print(model['sdfsfsdfsadfs'])

class Word:
    def __init__(self, text, vector):
        self.text = text
        self.vector = vector


class Sentence:
    def __init__(self, word_list):
        self.word_list = word_list

    def len(self) -> int:
        return len(self.word_list)


def get_word_frequency(word_text, looktable):
    if word_text in looktable:
        return looktable[word_text]
    else:
        return 1.0


def sentence_to_vec(sentence_list: List[Sentence], embedding_size, looktable, a=1e-3):
    sentence_set = []
    for sentence in sentence_list:
        vs = np.zeros(embedding_size)  # add all word2vec values into one vector for the sentence
        sentence_length = sentence.len()
        for word in sentence.word_list:
            a_value = a / (a + get_word_frequency(word.text, looktable))  # smooth inverse frequency, SIF
            vs = np.add(vs, np.multiply(a_value, word.vector))  # vs += sif * word_vector

        vs = np.divide(vs, sentence_length)  # weighted average
        sentence_set.append(vs)  # add to our existing re-calculated set of sentences

    # calculate PCA of this sentence set
    pca = PCA(n_components=embedding_size)
    pca.fit(np.array(sentence_set))
    u = pca.components_[0]  # the PCA vector
    u = np.multiply(u, np.transpose(u))  # u x uT

    # pad the vector?  (occurs if we have less sentences than embeddings_size)
    if len(u) < embedding_size:
        for i in range(embedding_size - len(u)):
            u = np.append(u, 0)  # add needed extension for multiplication below

    # resulting sentence vectors, vs = vs -u x uT x vs
    sentence_vecs = []
    for vs in sentence_set:
        sub = np.multiply(u, vs)
        sentence_vecs.append(np.subtract(vs, sub))

    return sentence_vecs


# with open(freq_table, 'rb') as f:
#     mydict = pkl.load(f)
# print('完成词频字典载入')

paths = ['./datasets/data']
for path in paths:
    files = []
    for file in os.listdir(path=path):
        if os.path.isfile(path + '/' + file):
            files.append(path + '/' + file)

    for traindata in files:
        with open(traindata, 'rb') as f:
            train = pkl.load(f)

        print('读取' + traindata + '数据完成')

        gs = []
        pred = []
        allsent = []
        for each in train:
            # sent1, sent2, label = each.split('\t')
            if len(train[0]) == 3:
                sent1, sent2, label = each
            else:
                sent1, sent2, label, _ = each
            gs.append(float(label))
            s1 = []
            s2 = []
            # sw1 = sent1.split()
            # sw2 = sent2.split()
            for word in sent1:
                try:
                    vec = model[word]
                except KeyError:
                    vec = np.zeros(embedding_size)
                s1.append(Word(word, vec))
            for word in sent2:
                try:
                    vec = model[word]
                except KeyError:
                    vec = np.zeros(embedding_size)
                s2.append(Word(word, vec))

            ss1 = Sentence(s1)
            ss2 = Sentence(s2)
            allsent.append(ss1)
            allsent.append(ss2)

        # sentence_vectors = sentence_to_vec(allsent, embedding_size, looktable=mydict)
        sentence_vectors = sentence_to_vec(allsent, embedding_size)
        len_sentences = len(sentence_vectors)
        for i in range(len_sentences):
            if i % 2 == 0:
                sim = cosine_similarity([sentence_vectors[i]], [sentence_vectors[i + 1]])
                pred.append(sim[0][0])

        print('len of pred: ', len(pred))
        print('len of gs: ', len(gs))

        r, p = pearsonr(pred, gs)
        print(traindata + '皮尔逊相关系数:', r)


        # sentence_vectors = sentence_to_vec([ss1, ss2], embedding_size, looktable=mydict)
        # sim = cosine_similarity([sentence_vectors[0]], [sentence_vectors[1]])
        # pred.append(sim[0][0])

        # r, p = pearsonr(pred, gs)
        # print(traindata + '皮尔逊相关系数:', r)  # print(sentence_vectors[0])
# print(sentence_vectors[1])


### 总结

这种句子的建模方式非常高效且便捷。由于这是一种无监督学习，那么就可以对大规模的语料加以利用，这是该方法相比于一般有监督学习的一大优势。 

通过对实验的复现，发现运行一次程序只需要十几分钟，并且主要的运行耗时都在将词向量模型载入内存这个过程中，这比动不动就需要训练几周的神经网络模型确实要好很多，并且在这个词相似性任务中，与神经网络旗鼓相当。